In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import SGD

In [14]:
# Define the path to your dataset
train_data_dir = 'Dataset\Preprocessed\Tomato\\train_set'
test_data_dir = 'Dataset\Preprocessed\Tomato\\test_set'
img_width, img_height = 400, 400
batch_size = 16

In [15]:
# Since the data is already pre-processed and augmented, we'll only apply the necessary preprocessing
datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

test_generator = datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

Found 21723 images belonging to 5 classes.
Found 5445 images belonging to 5 classes.


In [16]:
# Load the ResNet50 model, pretrained on ImageNet
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

In [17]:
# Add new layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(5, activation='softmax')(x)  # Assuming 5 classes

# This is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze all convolutional ResNet50 layers
for layer in base_model.layers:
    layer.trainable = False

In [18]:
# Compile the model
model.compile(optimizer=SGD(learning_rate=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])


In [20]:
# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=10,  # You can adjust epochs based on your requirement
    validation_data=test_generator,
    validation_steps=test_generator.samples // batch_size)

# Save the trained model
model.save('tomato_leaf_model_tf.h5')

Epoch 1/10


c:\Users\jagha\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


 616/1357 ━━━━━━━━━━━━━━━━━━━━ 17:27 1s/step - accuracy: 0.4353 - loss: 1.3680

KeyboardInterrupt: 